# Project: Constant Therapy ESP Toy Project
**Project Description**: Playing with ESP for Constant Therapy  
**Experiment**: run for notebook and others  
**Run ID**: 4522  
**Notebook**: Neuro AI - Models Training

In [1]:
# Uncomment the following line and execute the cell to install this notebook's dependencies.
# You might need to restart the notebook's kernel.

# %pip install -r project_Constant_Therapy_ESP_Toy_Project_experiment_run_for_notebook_and_others_run_4522_requirements.txt

In [2]:
import joblib
import numbers
import os
import matplotlib.pyplot as plt
import numpy as np
from onnxruntime import InferenceSession
import pandas as pd
from abc import ABC
from abc import abstractmethod
from io import StringIO
from pandas.api.types import is_numeric_dtype
from typing import Any
from typing import Dict
from typing import List
from typing import Optional
from typing import Tuple
from typing import Type
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from leaf_common.candidates.representation_types import RepresentationType
from leaf_common.representation.rule_based.config.rule_set_config_helper import RuleSetConfigHelper
from leaf_common.representation.rule_based.data.features import Features
from leaf_common.representation.rule_based.data.rule_set import RuleSet
from leaf_common.representation.rule_based.data.rule_set_binding import RuleSetBinding
from leaf_common.representation.rule_based.evaluation.rule_set_binding_evaluator import RuleSetBindingEvaluator
from leaf_common.representation.rule_based.persistence.rule_set_file_persistence import RuleSetFilePersistence
from esp_sdk.esp_evaluator import EspEvaluator
from esp_sdk.esp_service import EspService
from unileaf_util.framework.data.profiling.dataframe_profiler import DataFrameProfiler
from unileaf_util.framework.interfaces.data_frame_predictor import DataFramePredictor
from unileaf_util.framework.metrics.metrics_manager import MetricsManager
from unileaf_util.framework.transformers.data_encoder import DataEncoder
from unileaf_util.framework.transformers.rules_data_encoder import RulesDataEncoder

pd.set_option('display.max_columns', None)

## Load the dataset
By default, load the dataset exported with the notebook, but you may plug your own dataset by changing the path for DATASET_CSV here.

In [3]:
# Path to the dataset csv file
DATASET_CSV = 'esp_toy.csv'
with open(DATASET_CSV) as df_file:
    data_source_df = pd.read_csv(df_file)
data_source_df.head()

,Unnamed: 0,target_d_1,target_d_2,target_d_3,target_d_4,target_d_5,target_d_6,target_d_7,target_d_8,target_d_9,target_d_10,target_d_11,target_d_12,target_d_13,target_d_14,d_1_score,d_1_ind,d_2_score,d_2_ind,d_3_score,d_3_ind,d_4_score,d_4_ind,d_5_score,d_5_ind,d_6_score,d_6_ind,d_7_score,d_7_ind,d_8_score,d_8_ind,d_9_score,d_9_ind,d_10_score,d_10_ind,d_11_score,d_11_ind,d_12_score,d_12_ind,d_13_score,d_13_ind,d_14_score,d_14_ind,d_1_next,d_2_next,d_3_next,d_4_next,d_5_next,d_6_next,d_7_next,d_8_next,d_9_next,d_10_next,d_11_next,d_12_next,d_13_next,d_14_next
0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data_source_df.describe()

,Unnamed: 0,target_d_1,target_d_2,target_d_3,target_d_4,target_d_5,target_d_6,target_d_7,target_d_8,target_d_9,target_d_10,target_d_11,target_d_12,target_d_13,target_d_14,d_1_score,d_1_ind,d_2_score,d_2_ind,d_3_score,d_3_ind,d_4_score,d_4_ind,d_5_score,d_5_ind,d_6_score,d_6_ind,d_7_score,d_7_ind,d_8_score,d_8_ind,d_9_score,d_9_ind,d_10_score,d_10_ind,d_11_score,d_11_ind,d_12_score,d_12_ind,d_13_score,d_13_ind,d_14_score,d_14_ind,d_1_next,d_2_next,d_3_next,d_4_next,d_5_next,d_6_next,d_7_next,d_8_next,d_9_next,d_10_next,d_11_next,d_12_next,d_13_next,d_14_next
count,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
mean,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
std,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000
50%,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
75%,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.7

## Encode the dataset
Encode the dataset using the fields definition from the Experiment's data source

In [5]:
profiler = DataFrameProfiler()
data_profile = profiler.profile_data_frame(data_source_df)

# Get fields from the data profile
fields = data_profile.get('info')

In [6]:
import IPython.display
IPython.display.JSON(fields)

<IPython.core.display.JSON object>

In [7]:
cao_mapping = {'context': ['d_1_ind', 'd_2_ind', 'd_3_ind', 'd_4_ind', 'd_5_ind', 'd_6_ind', 'd_7_ind', 'd_8_ind', 'd_9_ind', 'd_10_ind', 'd_11_ind', 'd_12_ind', 'd_13_ind', 'd_14_ind', 'd_1_score', 'd_2_score', 'd_3_score', 'd_4_score', 'd_5_score', 'd_6_score', 'd_7_score', 'd_8_score', 'd_9_score', 'd_10_score', 'd_11_score', 'd_12_score', 'd_13_score', 'd_14_score'], 'actions': ['target_d_1', 'target_d_2', 'target_d_3', 'target_d_4', 'target_d_5', 'target_d_6', 'target_d_7', 'target_d_8', 'target_d_9', 'target_d_10', 'target_d_11', 'target_d_12', 'target_d_13', 'target_d_14'], 'outcomes': ['d_1_next', 'd_2_next', 'd_3_next', 'd_4_next', 'd_5_next', 'd_6_next', 'd_7_next', 'd_8_next', 'd_9_next', 'd_10_next', 'd_11_next', 'd_12_next', 'd_13_next', 'd_14_next']}
cao_fields =  set(cao_mapping['context'] + cao_mapping['actions'] + cao_mapping['outcomes'])

# Validate if the fields match with cao_mapping
missing_fields = set(fields.keys()) - cao_fields
extra_fields =  cao_fields - set(fields.keys())
if missing_fields != set():
    print(f'The dataset contains fields that are NOT part of cao_mapping: {missing_fields}')
    print('Please add them to the cao_mapping dictionary and make sure the rest of the notebook handles them correctly.')
if extra_fields != set():
    print(f'The cao_mapping contains fields that are NOT part of the dataset: {extra_fields}')
    print('Please remove them from the cao_mapping dictionary and make sure they are not used in the rest of the notebook.')

In [8]:
import IPython.display
IPython.display.JSON(cao_mapping)

<IPython.core.display.JSON object>

In [9]:
encoder = DataEncoder(fields, cao_mapping)
encoded_data_source_df = encoder.encode_as_df(data_source_df)
encoded_data_source_df.head()

,target_d_1,target_d_2,target_d_3,target_d_4,target_d_5,target_d_6,target_d_7,target_d_8,target_d_9,target_d_10,target_d_11,target_d_12,target_d_13,target_d_14,d_1_score,d_1_ind,d_2_score,d_2_ind,d_3_score,d_3_ind,d_4_score,d_4_ind,d_5_score,d_5_ind,d_6_score,d_6_ind,d_7_score,d_7_ind,d_8_score,d_8_ind,d_9_score,d_9_ind,d_10_score,d_10_ind,d_11_score,d_11_ind,d_12_score,d_12_ind,d_13_score,d_13_ind,d_14_score,d_14_ind,d_1_next,d_2_next,d_3_next,d_4_next,d_5_next,d_6_next,d_7_next,d_8_next,d_9_next,d_10_next,d_11_next,d_12_next,d_13_next,d_14_next
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Predictor code

In [10]:
REGRESSOR = 'regressor'
CLASSIFIER = 'classifier'
TYPES = [REGRESSOR, CLASSIFIER]
predictors_by_id = {}
model_metrics_by_id = {}

In [11]:
class PredictorType:
    """
    This class defines the type of Predictor Possible.
    """

    def __init__(self, predictor_type: str):
        """
        The constructor confirms if the type of predictor is supported.
        :param predictor_type: String describing a name for the type of the
        predictor.
        """
        assert predictor_type in TYPES, "Invalid Predictor Type"
        self.type = predictor_type

    def __str__(self) -> str:
        """
        This function overrides the string representation of the
        class.
        :return self.type: String
        """
        return self.type


In [12]:
class Predictor(DataFramePredictor, ABC):
    """
    This class contains the contract that any predictor
    must implement.
    """

    def __init__(self,
                 data_df: pd.DataFrame,
                 cao_mapping: Dict[str, List[str]],
                 data_split: Dict[str, float],
                 model_params: Dict = None,
                 metadata: Dict = None):
        """
        Initializes a predictor, its params and the metadata.
        :param data_df: DataFrame containing all processed data
        :param cao_mapping: a dictionary with `context`, `actions` and `outcomes`
        keys where each key returns a List of the selected column names as strings.
        :param data_split: Dictionary containing the training splits indexed
        by "train_pct" and "val_pct".
        :param model_params: Parameters of the model
        :param metadata: Dictionary describing any other information
        that must be stored along with the model.
        This might help in uniquely identifying the model
        :returns nothing
        """
        # Split the data between train, val and test sets
        self.data_split = data_split

        self.cao_mapping = cao_mapping
        self.context_actions_columns = self.cao_mapping["context"] + self.cao_mapping["actions"]
        # Check
        if len(cao_mapping["outcomes"]) > 1:
            if not self.does_support_multiobjective():
                raise ValueError(f"{self.predictor_name} does NOT support multiple outputs")

        self.column_length = {}
        if data_df is not None:
            train_df, val_df, test_df = self.generate_data_split(data_df, self.data_split)

            # Split the data between features (x) and labels(y)
            self.train_x_df, self.train_y_df = self.get_data_xy_split(train_df, cao_mapping)
            self.val_x_df, self.val_y_df = self.get_data_xy_split(val_df, cao_mapping)
            self.test_x_df, self.test_y_df = self.get_data_xy_split(test_df, cao_mapping)

            # Keep track of how many values are used to encode each outcome
            for column in self.cao_mapping["outcomes"]:
                first_value = self.train_y_df[column].head(1).values[0]
                if isinstance(first_value, numbers.Number):
                    # Value is a single scalar
                    self.column_length[column] = 1
                else:
                    # value is a one-hot encoded vector, i.e. a list. Get its size.
                    self.column_length[column] = len(self.train_y_df[column].head(1).values[0])
        else:
            # No data provided, assuming outcomes are numerical (not categorical)
            for column in self.cao_mapping["outcomes"]:
                self.column_length[column] = 1

        if model_params is None:
            model_params = {}
        self.model_params = model_params

        if metadata is None:
            metadata = {}
        self.metadata = metadata

        # Internal Parameters that are used to store the
        # latest state of the model.
        self._trained_model = None

    @property
    @abstractmethod
    def predictor_type(self) -> PredictorType:
        """
        :return the PredictorType of this Predictor: Regressor or Classifier
        """

    @property
    @abstractmethod
    def library(self) -> str:
        """
        :return the underlying library that implements this predictor, as a string
        """

    @property
    @abstractmethod
    def predictor_name(self) -> str:
        """
        :return: the name of the Predictor, as a string
        """

    @staticmethod
    @abstractmethod
    def does_support_multiobjective() -> bool:
        """
        This function returns if the predictor supports multiple outputs
        or not.
        :return multioutput: Bool
        """

    @abstractmethod
    def build_model(self, model_params: Dict):
        """
        This function must be overridden to build the model using the model
        parameters if desired and return a model.
        :param model_params: Dictionary containing the model parameters
        :return model: The built model.
        """

    @abstractmethod
    def train_model(self, model,
                    train_x: np.ndarray, train_y: np.ndarray,
                    val_x: Optional[np.ndarray], val_y: Optional[np.ndarray]) -> Type:
        """
        This function must be overridden to train the built model from the build_model step
        given the Data and must return the trained model.
        :param model: The model built in the build_model step
        :param train_x: numpy array containing the processed input features split for training
        :param train_y: numpy array containing the processed output features split for training
        :param val_x: Optional numpy array containing the processed input features split for validation
        :param val_y: Optional numpy array containing the processed output features split for validation

        :return trained_model
        """

    def set_trained_model(self, trained_model) -> None:
        """
        Sets the underlying trained model to the passed one.
        :param trained_model: a trained model
        :return Nothing:
        """
        self._trained_model = trained_model

    def get_trained_model(self):
        """
        Returns the trained model if it has been set, None otherwise
        :return self._trained_model:
        """
        return self._trained_model

    @staticmethod
    def generate_data_split(data_df: pd.DataFrame,
                            data_split: Dict[str, Any]) -> Tuple[pd.DataFrame, Optional[pd.DataFrame], pd.DataFrame]:
        """
        Splits the data between train, validation (optional) and test sets
        :param data_df: the full dataset as a Pandas DataFrame
        :param data_split: a dictionary with the
        :return: a tuple of Pandas DataFrame: one for train, one for validation (or None), and one for test
        """

        # First, split the data set in train and test sets.
        # Use the provided random_state, if any
        random_state = data_split.get("random_state", None)
        shuffle = data_split.get("shuffle", True)
        train_df, test_df = train_test_split(data_df,
                                             test_size=data_split["test_pct"],
                                             random_state=random_state,
                                             shuffle=shuffle)

        # If we also need a validation set, split the train set into train and validation sets.
        val_pct = data_split.get("val_pct", 0)
        if val_pct > 0:
            train_df, val_df = train_test_split(train_df,
                                                test_size=val_pct,
                                                random_state=random_state,
                                                shuffle=shuffle)
        else:
            val_df = None
        return train_df, val_df, test_df

    def predict(self, encoded_context_actions_df: pd.DataFrame) -> pd.DataFrame:
        """
        This method uses the trained model to make a prediction for the passed Pandas DataFrame
        of context and actions. Returns the predicted outcomes in a Pandas DataFrame.
        :param encoded_context_actions_df: a Pandas DataFrame containing encoded rows of context and actions for
        which a prediction is requested. Categorical columns contain one-hot vectors, e.g. [1, 0, 0]. Which means
        a row can be a list of arrays (1 per column), e.g.: [1, 0, 0], [1,0].
        :return a Pandas DataFrame of the predicted outcomes for each context and actions row.
        """
        # Default implementation
        if self._trained_model:
            # Predict using the model's input columns, in case encoded_context_actions_df contains more columns
            # or is in a different order
            context_action_df = encoded_context_actions_df[self.context_actions_columns]
            # Convert one-hot vector columns into a single feature vector
            features = DataEncoder.encoded_df_to_np(context_action_df)
            # Check if model type is onnx runtime or not
            if isinstance(self._trained_model, InferenceSession):
                predictions = self._trained_model.run(None, {"X": features.astype(np.float32)})[0]
            else:
                predictions = self._trained_model.predict(features)
            if isinstance(predictions, pd.DataFrame):
                # Predictions are already in a DataFrame. Make sure they have the correct outcome names
                predictions_df = predictions
                predictions_df.columns = self.cao_mapping["outcomes"]
                # Convert predictions to float64 as it's JSON serializable, while float32 is not
                predictions_df = predictions_df.astype("float64")
            else:
                # Assuming predictions is a ndarray, convert it to a DataFrame with the output column names
                predictions_df = DataEncoder.np_to_encoded_df(predictions,
                                                              self.column_length)
        else:
            raise ValueError("Can't make predictions because the model has not been trained")
        return predictions_df

    @staticmethod
    def export_metrics(metrics_dict: Dict[str, Any], file_path: str):
        """
        Save the model's training metrics to the specified location
        :param metrics_dict: a dictionary containing metrics
        :param file_path: the name and path of the file to persist the bytes to
        :return: nothing
        """
        with open(file_path, 'w', encoding='utf-8') as my_file:
            json.dump(metrics_dict, my_file)

    @staticmethod
    def get_data_xy_split(data_df: Optional[pd.DataFrame],
                          cao_mapping: Dict) -> Tuple[Optional[pd.DataFrame], Optional[pd.DataFrame]]:
        """
        This function takes a dataframe and a dictionary mapping indices to context,
        action, or outcome. This then splits the dataframe into two dataframes based
        on it's CAO tagging.

        data_x: Context and Actions
        data_y: Outcomes

        :param data_df: a Pandas DataFrame with all the data
        :param cao_mapping: a dictionary with `context`, `actions` and `outcomes` keys where each key returns a List
         ofthe selected column names as strings.
        :return: A tuple containing two dataframes: data_x with the features, and data_y with the labels (outcomes)
        """
        if data_df is None:
            return None, None

        data_x_df = data_df[cao_mapping["context"] + cao_mapping["actions"]]
        data_y_df = data_df[cao_mapping["outcomes"]]

        return data_x_df, data_y_df

    def __str__(self):
        return self.predictor_name


In [13]:
class Regressor(PredictorType):
    """
    This class defines a Regressor Type
    """

    def __init__(self):
        """
        The constructor initializes the super class.
        """
        super().__init__(REGRESSOR)


In [14]:
class Classifier(PredictorType):
    """
    This class defines a Classifier Type
    """

    def __init__(self):
        """
        The constructor initializes the super class.
        """
        super().__init__(CLASSIFIER)


## Predictor 704abc67-fe1c-409b-87c1-8e59864b7fe4
### CAO columns

In [15]:
CONTEXT_COLUMNS = ['d_1_ind', 'd_2_ind', 'd_3_ind', 'd_4_ind', 'd_5_ind', 'd_6_ind', 'd_7_ind', 'd_8_ind', 'd_9_ind', 'd_10_ind', 'd_11_ind', 'd_12_ind', 'd_13_ind', 'd_14_ind', 'd_1_score', 'd_2_score', 'd_3_score', 'd_4_score', 'd_5_score', 'd_6_score', 'd_7_score', 'd_8_score', 'd_9_score', 'd_10_score', 'd_11_score', 'd_12_score', 'd_13_score', 'd_14_score']
ACTION_COLUMNS = ['target_d_1', 'target_d_2', 'target_d_3', 'target_d_4', 'target_d_5', 'target_d_6', 'target_d_7', 'target_d_8', 'target_d_9', 'target_d_10', 'target_d_11', 'target_d_12', 'target_d_13', 'target_d_14']
OUTCOME_COLUMNS = ['d_3_next']
CONTEXT_ACTION_COLUMNS = CONTEXT_COLUMNS + ACTION_COLUMNS

### Data split

In [16]:
data_split = {"train_pct": 0.8, "test_pct": 0.2, "random_state": 42}

### Predictor code

In [17]:
class LinearRegression(Predictor):
    """
    This class implements a linear regression model from the SKLearn library.
    """
    predictor_type = Regressor()
    library = "sklearn"
    predictor_name = name = f"{library} Linear Regression"

    def __init__(self,
                 data_df: pd.DataFrame,
                 cao_mapping: Dict[str, List[str]],
                 data_split: Dict = None,
                 model_params: Dict = None,
                 metadata: Dict = None):
        """
        The constructor initializes the base params.
        """
        super().__init__(data_df=data_df,
                         cao_mapping=cao_mapping,
                         data_split=data_split,
                         model_params=model_params,
                         metadata=metadata)

    @staticmethod
    def does_support_multiobjective() -> bool:
        """
        This function returns if the predictor supports multiple outputs
        or not.
        :return multioutput: Bool
        """
        multioutput = True
        return multioutput

    def build_model(self, model_params: Dict[str, Any]) -> linear_model.LinearRegression:
        """
        This function instantiates a Linear Regression model with the given params.
        :return model: a LinearRegression instance
        """
        model = linear_model.LinearRegression(**model_params)
        return model

    def train_model(self, model: linear_model.LinearRegression,
                    train_x: np.ndarray, train_y: np.ndarray,
                    val_x: Optional[np.ndarray], val_y: Optional[np.ndarray])\
            -> linear_model.LinearRegression:
        """
        This function must be overridden to train the built model from the build_model step
        given the Data and must return the trained model and the desired metrics as a dictionary.
        :param model: The model built in the build_model step
        :param train_x: numpy array containing the processed input features split for training
        :param train_y: numpy array containing the processed output features split for training
        :param val_x: Optional numpy array containing the processed input features split for validation
        :param val_y: Optional numpy array containing the processed output features split for validation

        :return trained_model: The linear regression model trained
        """
        trained_model = model.fit(train_x, train_y)
        return trained_model


In [18]:
predictor_node_id = '704abc67-fe1c-409b-87c1-8e59864b7fe4'
predictor = LinearRegression(encoded_data_source_df,
    cao_mapping={'context': ['d_1_ind', 'd_2_ind', 'd_3_ind', 'd_4_ind', 'd_5_ind', 'd_6_ind', 'd_7_ind', 'd_8_ind', 'd_9_ind', 'd_10_ind', 'd_11_ind', 'd_12_ind', 'd_13_ind', 'd_14_ind', 'd_1_score', 'd_2_score', 'd_3_score', 'd_4_score', 'd_5_score', 'd_6_score', 'd_7_score', 'd_8_score', 'd_9_score', 'd_10_score', 'd_11_score', 'd_12_score', 'd_13_score', 'd_14_score'], 'actions': ['target_d_1', 'target_d_2', 'target_d_3', 'target_d_4', 'target_d_5', 'target_d_6', 'target_d_7', 'target_d_8', 'target_d_9', 'target_d_10', 'target_d_11', 'target_d_12', 'target_d_13', 'target_d_14'], 'outcomes': ['d_3_next']},
data_split=data_split,
model_params={},
metadata={})

### Train Predictor

In [19]:
train_x = DataEncoder.encoded_df_to_np(predictor.train_x_df)
train_y = DataEncoder.encoded_df_to_np(predictor.train_y_df)
if predictor.val_x_df is not None:
    val_x = DataEncoder.encoded_df_to_np(predictor.val_x_df)
    val_y = DataEncoder.encoded_df_to_np(predictor.val_y_df)
else:
    val_x = None
    val_y = None

In [20]:
# Set train to True to train a new model, False to re-use a previously trained one
train=True
if train:
    model = predictor.build_model(predictor.model_params)
    trained_model = predictor.train_model(model,
                                          train_x, train_y,
                                          val_x, val_y)
    joblib.dump(trained_model, 'predictor-704abc67-fe1c-409b-87c1-8e59864b7fe4.joblib')
    predictor.set_trained_model(trained_model)
else:
    trained_model = joblib.load('predictor-704abc67-fe1c-409b-87c1-8e59864b7fe4.joblib')
    predictor.set_trained_model(trained_model)
predictors_by_id[predictor_node_id] = predictor

### Predictor metrics

In [21]:
model_metrics_by_id[predictor_node_id] = [MetricsManager.get_calculator('Mean Squared Error')]
metrics = MetricsManager.compute_metrics(predictor,
model_metrics_by_id[predictor_node_id],predictor.train_x_df, predictor.train_y_df,predictor.val_x_df, predictor.val_y_df,predictor.test_x_df, predictor.test_y_df,encoder)

print(f'Predictor trained. Metrics: {metrics}')

Predictor trained. Metrics: {'train_Mean Squared Error_d_3_next': 0.0, 'test_Mean Squared Error_d_3_next': 1.0}


## Prescriptor

### Evaluation code

In [22]:
class UnileafPrescriptor(EspEvaluator):
    """
    An Unileaf Prescriptor makes prescriptions given an ESP candidate and a context DataFrame.
    It is also an EspEvaluator implementation that returns metrics for ESP candidates.
    """

    def __init__(self,
                 config: Dict[str, Any],
                 evaluation_df: pd.DataFrame,
                 data_encoder: DataEncoder,
                 predictors: List[Predictor]):
        """
        Constructs a prescriptor evaluator
        :param config: the ESP experiment config dictionary
        :param evaluation_df: the Pandas DataFrame to use to evaluate the candidates
        :param data_encoder: the DataEncoder used to encode the dataset
        :param predictors: the predictors this prescriptor relies on
        """
        # Instantiate EspEvaluator
        # Note: sets self.config
        super().__init__(config)

        # CAO
        self.cao_mapping = {"context": self.get_context_field_names(config),
                            "actions": self.get_action_field_names(config),
                            "outcomes": self.get_fitness_metrics(config)}
        self.context_df = evaluation_df[self.cao_mapping["context"]]
        self.row_index = self.context_df.index

        # Convert the context DataFrame to a format a NN can ingest
        self.context_as_nn_input = self.convert_to_nn_input(self.context_df)

        # Data encoder
        self.data_encoder = data_encoder

        # Predictors
        self.predictors = predictors

    @staticmethod
    def convert_to_nn_input(context_df: pd.DataFrame) -> List[np.ndarray]:
        """
        Converts a context DataFrame to a list of numpy arrays a neural network can ingest
        :param context_df: a DataFrame containing inputs for a neural network. Number of inputs and size must match
        :return: a list of numpy ndarray, on ndarray per neural network input
        """
        # The NN expects a list of i inputs by s samples (e.g. 9 x 299).
        # So convert the data frame to a numpy array (gives shape 299 x 9), transpose it (gives 9 x 299)
        # and convert to list(list of 9 arrays of 299)
        context_as_nn_input = list(context_df.to_numpy().transpose())
        # Convert each column's list of 1D array to a 2D array
        context_as_nn_input = [np.stack(context_as_nn_input[i], axis=0) for i in
                               range(len(context_as_nn_input))]
        return context_as_nn_input

    def evaluate_candidate(self, candidate):
        """
        Evaluates a single Prescriptor candidate and returns its metrics.
        Implements the EspEvaluator interface
        :param candidate: a Keras neural network or rule based Prescriptor candidate
        :return metrics: A dictionary of {'metric_name': metric_value}
        """
        # Prescribe actions
        prescribed_actions_df = self.prescribe(candidate)

        # Aggregate the context and actions dataframes.
        context_actions_df = pd.concat([self.context_df,
                                        prescribed_actions_df],
                                       axis=1)

        # Compute the metrics
        metrics = self._compute_metrics(context_actions_df)
        return metrics

    def _compute_metrics(self, context_actions_df):
        """
        Computes metrics from the passed context/actions DataFrame using the instance's trained predictors.
        :param context_actions_df: a DataFrame of context / prescribed actions
        :return: A dictionary of {'metric_name': metric_value}
        """
        # Get the predicted outcomes from the predictors
        metrics = {}
        for predictor in self.predictors:
            predicted_outcomes = predictor.predict(context_actions_df)

            # UN-853: Decode predictions before computing numerical metrics, if a data_encoder is available
            if self.data_encoder is not None:
                decoded_predicted_outcomes = self.data_encoder.decode_as_df(predicted_outcomes)
            else:
                decoded_predicted_outcomes = predicted_outcomes

            # Only add a metric for the outcomes the prescriptor is interested in
            for outcome in self.cao_mapping["outcomes"]:
                # Add the metrics that have been produced by this predictor
                if outcome in predictor.cao_mapping["outcomes"]:
                    # Check the type of metric: numerical or categorical?
                    if decoded_predicted_outcomes[[outcome]].iloc[:, 0].dtype == object:
                        # Categorical outcome. Use the *encoded* predicted outcome.
                        preds = predicted_outcomes[outcome]
                        # Classifiers return the category's index in the list of categories, so we can take the mean
                        # of the encoded outcomes. Note: this works because Outcomes are encoded using LabelEncoder
                        # AND the user defined order for each Outcome categories.
                        metrics[outcome] = preds.mean()
                    else:
                        # UN-853: Numerical outcome. Use the *decoded*, i.e. scaled back, predicted outcome
                        preds = decoded_predicted_outcomes[outcome]
                        # Regressors produce floats: take the mean of the decoded outcome
                        metrics[outcome] = preds.mean()
        return metrics

    def prescribe(self, candidate, context_df: pd.DataFrame = None) -> pd.DataFrame:
        """
        Generates prescriptions using the passed candidate and context
        :param candidate: an ESP candidate, either neural network or rules
        :param context_df: a DataFrame containing the context to prescribe for,
         or None to use the instance one
        :return: a DataFrame containing actions prescribed for each context
        """
        if context_df is None:
            # No context is provided, use the instance's one
            context_as_nn_input = self.context_as_nn_input
            row_index = self.row_index
        else:
            # Convert the context DataFrame to something more suitable for neural networks
            context_as_nn_input = self.convert_to_nn_input(context_df)
            # Use the context's row index
            row_index = context_df.index

        is_rule_based = isinstance(candidate, RuleSet)
        if is_rule_based:
            actions = self._prescribe_from_rules(candidate, context_as_nn_input)
        else:
            actions = self._prescribe_from_nn(candidate, context_as_nn_input)

        # Convert the prescribed actions to a DataFrame
        prescribed_actions_df = pd.DataFrame(actions,
                                             columns=self.cao_mapping["actions"],
                                             index=row_index)
        ### DEBUG:
        print("-------------DEBUG: Prescribed Actions (encoded):")
        print("shape: ", prescribed_actions_df.shape)
        print("dtype: ", prescribed_actions_df.dtypes)
        has_arrays = prescribed_actions_df.applymap(lambda x: isinstance(x, (list, np.ndarray))).any() # type: ignore
        print("array-valued columns:\n", has_arrays[has_arrays].index.tolist())

        # UN-2430 Decode the softmaxes, if any, back into categories
        prescribed_actions_df = self.data_encoder.decode_as_df(prescribed_actions_df)
        # UN0-240 Re-encode the actions into what the predictors expect (e.g. one-hots for categorical data)
        prescribed_actions_df = self.data_encoder.encode_as_df(prescribed_actions_df)
        return prescribed_actions_df

    def _prescribe_from_rules(self, candidate, context_as_nn_input: List[np.ndarray]):
        """
        Generates prescriptions using the passed rules model candidate and context
        :param candidate: a rules model candidate
        :param context_as_nn_input: a numpy array containing the context to prescribe for
        :return: a dictionary of action name to list of action values
        """
        cand_states = RuleSetConfigHelper.get_states(self.config)
        cand_actions = RuleSetConfigHelper.get_actions(self.config)
        candidate = RuleSetBinding(candidate, cand_states, cand_actions)
        rules_encoder = RulesDataEncoder(candidate.actions)
        evaluator = RuleSetBindingEvaluator()
        rules_input = rules_encoder.encode_to_rules_data(context_as_nn_input)
        rules_output = evaluator.evaluate(candidate, rules_input)
        actions = rules_encoder.decode_from_rules_data(rules_output)
        return actions

    def _prescribe_from_nn(self, candidate, context_as_nn_input: List[np.ndarray]) -> Dict[str, Any]:
        """
        Generates prescriptions using the passed neural network candidate and context
        :param candidate: a Keras neural network candidate
        :param context_as_nn_input: a numpy array containing the context to prescribe for
        :return: a dictionary of action name to action value or list of action values
        """
        # Get the prescribed actions
        prescribed_actions = candidate.predict(context_as_nn_input)
        actions = {}

        if self._is_single_action_prescriptor():
            # Put the single action in an array to process it like multiple actions
            prescribed_actions = [prescribed_actions]

        for index, action_col in enumerate(self.cao_mapping["actions"]):
            if self._is_scalar(prescribed_actions[index]):
                # We have a single row and this action is numerical. Convert it to a scalar.
                actions[action_col] = prescribed_actions[index].item()
            else:
                actions[action_col] = prescribed_actions[index].tolist()
        return actions

    def _is_single_action_prescriptor(self):
        """
        Checks how many Actions have been defined in the Context, Actions, Outcomes mapping.
        :return: True if only 1 action is defined, False otherwise
        """
        return len(self.cao_mapping["actions"]) == 1

    @staticmethod
    def _is_scalar(prescribed_action):
        """
        Checks if the prescribed action contains a single value, i.e. a scalar, or an array.
        A prescribed action contains a single value if it has been prescribed for a single context sample
        :param prescribed_action: a scalar or an array
        :return: True if the prescribed action contains a scalar, False otherwise.
        """
        return prescribed_action.shape[0] == 1 and prescribed_action.shape[1] == 1

    @staticmethod
    def get_context_field_names(config: Dict[str, Any]) -> List[str]:
        """
        Returns the list of Context column names
        :param config: the ESP experiment config dictionary
        :return: the list of Context column names
        """
        nn_inputs = config["network"]["inputs"]
        contexts = [nn_input["name"] for nn_input in nn_inputs]
        return contexts

    @staticmethod
    def get_action_field_names(config: Dict[str, Any]) -> List[str]:
        """
        Returns the list of Action column names
        :param config: the ESP experiment config dictionary
        :return: the list of Action column names
        """
        nn_outputs = config["network"]["outputs"]
        actions = [nn_output["name"] for nn_output in nn_outputs]
        return actions

    @staticmethod
    def get_fitness_metrics(config: Dict[str, Any]) -> List[str]:
        """
        Returns the list of fitness metric names (Outcomes) to optimize.
        :param config: the ESP experiment config dictionary
        :return: the list of fitness metric names
        """
        metrics = config["evolution"]["fitness"]
        fitness_metrics = [metric["metric_name"] for metric in metrics]
        return fitness_metrics


### Prescriptor training

In [23]:
config = {'evolution': {'fitness': [{'maximize': True, 'metric_name': 'd_3_next'}], 'nb_elites': 5, 'mutation_type': 'gaussian_noise_percentage', 'nb_generations': 40, 'mutation_factor': 0.1, 'population_size': 10, 'parent_selection': 'tournament', 'initialization_range': 1, 'mutation_probability': 0.1, 'remove_population_pct': 0.8, 'initialization_distribution': 'orthogonal'}, 'network': {'inputs': [{'name': 'd_1_ind', 'size': 1, 'values': ['float']}, {'name': 'd_2_ind', 'size': 1, 'values': ['float']}, {'name': 'd_3_ind', 'size': 1, 'values': ['float']}, {'name': 'd_4_ind', 'size': 1, 'values': ['float']}, {'name': 'd_5_ind', 'size': 1, 'values': ['float']}, {'name': 'd_6_ind', 'size': 1, 'values': ['float']}, {'name': 'd_7_ind', 'size': 1, 'values': ['float']}, {'name': 'd_8_ind', 'size': 1, 'values': ['float']}, {'name': 'd_9_ind', 'size': 1, 'values': ['float']}, {'name': 'd_10_ind', 'size': 1, 'values': ['float']}, {'name': 'd_11_ind', 'size': 1, 'values': ['float']}, {'name': 'd_12_ind', 'size': 1, 'values': ['float']}, {'name': 'd_13_ind', 'size': 1, 'values': ['float']}, {'name': 'd_14_ind', 'size': 1, 'values': ['float']}, {'name': 'd_1_score', 'size': 1, 'values': ['float']}, {'name': 'd_2_score', 'size': 1, 'values': ['float']}, {'name': 'd_3_score', 'size': 1, 'values': ['float']}, {'name': 'd_4_score', 'size': 1, 'values': ['float']}, {'name': 'd_5_score', 'size': 1, 'values': ['float']}, {'name': 'd_6_score', 'size': 1, 'values': ['float']}, {'name': 'd_7_score', 'size': 1, 'values': ['float']}, {'name': 'd_8_score', 'size': 1, 'values': ['float']}, {'name': 'd_9_score', 'size': 1, 'values': ['float']}, {'name': 'd_10_score', 'size': 1, 'values': ['float']}, {'name': 'd_11_score', 'size': 1, 'values': ['float']}, {'name': 'd_12_score', 'size': 1, 'values': ['float']}, {'name': 'd_13_score', 'size': 1, 'values': ['float']}, {'name': 'd_14_score', 'size': 1, 'values': ['float']}], 'outputs': [{'name': 'target_d_1', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_2', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_3', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_4', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_5', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_6', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_7', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_8', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_9', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_10', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_11', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_12', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_13', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}, {'name': 'target_d_14', 'size': 2, 'activation': 'softmax', 'use_bias': True, 'values': ['0.0', '1.0']}], 'hidden_layers': [{'layer_name': 'hidden_1', 'layer_type': 'Dense', 'layer_params': {'units': 16, 'use_bias': True, 'activation': 'tanh'}}]}, 'LEAF': {'representation': 'NNWeights', 'experiment_id': 'UniLEAF_de95fcd7-7056-43d3-9e30-9d309e76f726', 'version': '1.0.0', 'persistence_dir': 'trained_prescriptors/', 'candidates_to_persist': 'best'}}

In [24]:
import IPython.display
IPython.display.JSON(config)

<IPython.core.display.JSON object>

In [25]:
required_predictor_ids = ['704abc67-fe1c-409b-87c1-8e59864b7fe4']
all_predictors = [predictors_by_id[required_id] for required_id in required_predictor_ids]

In [26]:
# Instantiate the EspService
esp_service = EspService(config)
esp_evaluator = UnileafPrescriptor(config,
                                   encoded_data_source_df,
                                   encoder,
                                   all_predictors)
experiment_results_dir = esp_service.train(esp_evaluator)

Starting training:
  experiment_id: UniLEAF_de95fcd7-7056-43d3-9e30-9d309e76f726
  checkpoint_id: None
  timestamp: 20251024-164626
Asking ESP for a seed generation...
Seed generation received.
Evaluating PopulationResponse for generation 1...:
PopulationResponse:
  Generation: 1
  Population size: 10
  Checkpoint id: UniLEAF_de95fcd7-7056-43d3-9e30-9d309e76f726/1/20251024-224626
Evaluating candidates synchronously because max_workers == 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
-------------DEBUG: Prescribed Actions (encoded):
shape:  (2, 14)
dtype:  target_d_1     object
target_d_2     object
target_d_3     object
target_d_4     object
target_d_5     object
target_d_6     object
target_d_7     object
target_d_8     object
target_d_9     object
target_d_10    object
target_d_11    object
target_d_12    object
target_d_13    object
target_d_14    object
dtype: object
array-valued columns:
 ['target_d_1', 'target_d_2', 'target_d_3', 'target_d_4', 'target_d_5', 'target_d_6', 'target_d_7',

/var/folders/sq/wczx8z_50yn5szt1q9b333fr0000gp/T/ipykernel_19589/1568703674.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  has_arrays = prescribed_actions_df.applymap(lambda x: isinstance(x, (list, np.ndarray))).any() # type: ignore


ValueError: Length of values (4) does not match length of index (2)

## ESP Summary Stats
esp_service.train(...) returned the directory in which the experiment results are stored

In [ ]:
stats_file = os.path.join(experiment_results_dir, 'experiment_stats.csv')
with open(stats_file) as csv_file:
    stats_df = pd.read_csv(csv_file, sep=',')
stats_df

## ESP Summary Plot
esp_service.train(...) generated a plot summarizing the experiment's progress

In [ ]:
from IPython.display import Image

### d_3_next

In [ ]:
plot_file = os.path.join(experiment_results_dir, 'd_3_next_plot.png')
Image(filename=plot_file)

# Models usage
## Load the Prescriptor

In [ ]:
# Use the last row of the stats DataFrame, i.e. the last generation, to find the best model
last_gen = stats_df['generation'].iloc[-1]
best_score = stats_df['max_d_3_next'].iloc[-1]
cid_best_score = stats_df['cid_max_d_3_next'].iloc[-1]
prescriptor_model_filename = os.path.join(experiment_results_dir,
                                          str(last_gen),
                                          cid_best_score + '.h5')
print(f'Best max_d_3_next average is {best_score:.3f} for candidate id {cid_best_score}')

In [ ]:
from keras.models import load_model

print(f'Loading prescriptor model: {prescriptor_model_filename}')
prescriptor_model = load_model(prescriptor_model_filename, compile=False)

## Get a sample context
Get the context from one of the rows in the dataset, and make a prescription for it.

In [ ]:
sample_df = data_source_df.sample(1)
sample_context_df = sample_df[CONTEXT_COLUMNS]
sample_context_action_df = sample_df[CONTEXT_ACTION_COLUMNS]
sample_df

### Prescribe


In [ ]:
encoded_sample_context_df = encoder.encode_as_df(sample_context_df)
encoded_prescribed_actions_df = esp_evaluator.prescribe(prescriptor_model, encoded_sample_context_df)

In [ ]:
# Aggregate the context and actions dataframes.
encoded_context_actions_df = pd.concat([encoded_sample_context_df,
                                        encoded_prescribed_actions_df],
                                       axis=1)
sample_context_prescribed_action_df = encoder.decode_as_df(encoded_context_actions_df)
sample_context_prescribed_action_df

### Make predictions


In [ ]:
def get_predictions(predictors, context_action_df, encoder):
    pred_array = []
    for predictor in predictors:
        pred = predictor.predict(encoder.encode_as_df(context_action_df))
        pred_array.append(encoder.decode_as_df(pred))
    preds_df = pd.concat(pred_array, axis=1)
    return preds_df

#### With original actions

In [ ]:
original_actions_preds = get_predictions(all_predictors, sample_context_action_df, encoder)
original_actions_preds

#### With prescribed actions

In [ ]:
prescribed_actions_preds = get_predictions(all_predictors, sample_context_prescribed_action_df, encoder)
prescribed_actions_preds

#### With custom actions

In [ ]:
sample_context_custom_action_df = sample_context_prescribed_action_df.copy()
 # TODO: Uncomment and replace by the name of the actions(s) to customize
# sample_context_custom_action_df['SOME_ACTION_TO_CUSTOMIZE'] = 42
sample_context_custom_action_df

In [ ]:
custom_actions_preds = get_predictions(all_predictors, sample_context_custom_action_df, encoder)
custom_actions_preds

### Compare
Compare 3 rows in a single table:
- the original sample
- the sample with the prescribed actions
- the sample with some custom actions

In [ ]:
OUTCOME_COLUMNS = list(original_actions_preds.columns)
OUTCOME_COLUMNS

In [ ]:
# Observed OUTCOMES for the sample
pd.DataFrame(sample_df[OUTCOME_COLUMNS])

In [ ]:
# Context and actions for 3 rows:
# - the original sample
# - the sample with the prescribed actions
# - the sample with some custom actions
comp_df = pd.concat([sample_context_action_df,
                     sample_context_prescribed_action_df,
                     sample_context_custom_action_df], axis=0)

# Compute the outcomes
outcomes_dict = {}
for outcome in OUTCOME_COLUMNS:
    # Observed outcome from the sample in the dataset
    outcomes_dict[outcome] = [sample_df[outcome].iloc[0],
                              sample_df[outcome].iloc[0],
                              sample_df[outcome].iloc[0]]
    # Predicted outcome
    outcomes_dict[f'{outcome}_predicted'] = [original_actions_preds[outcome].iloc[0],
                                             prescribed_actions_preds[outcome].iloc[0],
                                             custom_actions_preds[outcome].iloc[0]]
    # For numerical outcomes, compute the diff between predicted and observed
    if is_numeric_dtype(outcomes_dict[outcome][0]):
        diff = [a - b for a, b in zip(outcomes_dict[f'{outcome}_predicted'],
                                      outcomes_dict[outcome])]
        outcomes_dict[f'{outcome}_diff'] = diff
    
outcomes_df = pd.DataFrame(outcomes_dict)
comp_df[list(outcomes_dict.keys())] = outcomes_df[list(outcomes_dict.keys())].values
comp_df


## Initial state

In [ ]:
context = sample_df[CONTEXT_COLUMNS].to_dict('records')[0]
IPython.display.JSON(context)

In [ ]:
actions = sample_df[ACTION_COLUMNS].to_dict('records')[0]
IPython.display.JSON(actions)

In [ ]:
outcomes = sample_df[OUTCOME_COLUMNS].to_dict('records')[0]
IPython.display.JSON(outcomes)